In [ ]:
import os
from ultralytics import YOLO, trackers
import cv2
import os
import supervision as sv
import cv2
import numpy as np
from ultralytics.data.annotator import auto_annotate
import torch

In [19]:
import time
import torch
import cv2
import os
from ultralytics import YOLO

device = "cuda" if torch.cuda.is_available() else "cpu"

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

model_path = os.path.join('runs/detect/train12/weights', 'best.pt')
model = YOLO(model_path, verbose=False)

input_video_path = './test/input.mp4'
output_video_path = './test/output.mp4'

cap = cv2.VideoCapture(input_video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

out = cv2.VideoWriter(
    output_video_path,
    cv2.VideoWriter_fourcc(*'MP4V'),
    fps,
    (frame_width, frame_height)
)

frames = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frames.append(frame)

 
cap.release()
out.release()
cv2.destroyAllWindows()




In [ ]:
#Simple producer and consumer
#Demonstrates queue and event with locks

#Imports
import random
import threading
import multiprocessing
import logging
from threading import Thread
from queue import Queue
import time
from multiprocessing import Pool

import time

work = (["A", 5], ["B", 2], ["C", 1], ["D", 3])


def work_log(work_data):
    print(" Process %s waiting %s seconds" % (work_data[0], work_data[1]))
    time.sleep(int(work_data[1]))
    print(" Process %s Finished." % work_data[0])


def pool_handler():
    p = Pool(2)
    p.map(work_log, work)


pool_handler()

In [ ]:
import time
import torch
import cv2
import os
from ultralytics import YOLO

device = "cuda" if torch.cuda.is_available() else "cpu"

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

model_path = os.path.join('runs/detect/train12/weights', 'best.pt')
model = YOLO(model_path, verbose=False)

input_video_path = './test/input.mp4'
output_video_path = './test/output.mp4'

cap = cv2.VideoCapture(input_video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

out = cv2.VideoWriter(
    output_video_path,
    cv2.VideoWriter_fourcc(*'MP4V'),
    fps,
    (frame_width, frame_height)
)

inference_times = []  

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    start_time = time.time()  
    results = model(frame)
    end_time = time.time()  

    inference_time = (end_time - start_time) * 1000  
    inference_times.append(inference_time)

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0].item()
            cls = int(box.cls[0].item())
            label = f"{model.names[cls]} {conf:.2f}"

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    out.write(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

# Calculate mean inference time
if inference_times:
    mean_inference_time = sum(inference_times) / len(inference_times)
    print(f"Mean Inference Time: {mean_inference_time:.2f} ms per frame")
